In [ ]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [ ]:
#connecting colab to my drive ) so please add a folder named AI in your drive if you really want to run it 
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/

Mounted at /gdrive
/gdrive


In [1]:
#locating training and testing dataset in my drive
%cd My\ Drive
%cd AI
! ls

[Errno 2] No such file or directory: 'My Drive'
/content
[Errno 2] No such file or directory: 'IDS project'
/content
sample_data


In [ ]:
def merge(list1, list2): 
      
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))] 
    return merged_list 

In [ ]:
#download the training dataset from drive
f=open('train.txt', 'r')
train = f.read()

#download the test dataset from drive
f1=open('test.txt', 'r')
test = f1.read()

#cleaning the training data 
temp1=train.split()
temp2=temp1[::3]
#skipping every 3rd word
temp3=temp1[1::3]
train_tagged_words=merge(temp2,temp3)

#cleaning the testing data
temp1=test.split()
temp2=temp1[::3]
#skipping every 3rd word
temp3=temp1[1::3]
test_tagged_words=merge(temp2,temp3)

#printing the training data small
print(train_tagged_words[:2])

#train_data=list(train_data)

[('Confidence', 'NN'), ('in', 'IN')]


In [ ]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary not really needed for final 
#vocab = {word for word,tag in train_data}

44
{'RBR', 'NNPS', 'JJ', '$', 'NNP', 'MD', 'WP', ',', 'NN', 'VBG', ')', 'DT', ':', 'RB', 'JJR', 'IN', 'RP', 'VB', 'WP$', '``', 'CC', 'VBZ', 'FW', 'PRP$', 'WRB', 'JJS', 'TO', 'UH', 'EX', 'NNS', "''", 'PDT', '(', 'VBD', 'WDT', 'RBS', 'VBP', 'VBN', '.', 'SYM', '#', 'PRP', 'CD', 'POS'}


In [ ]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [ ]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[0.0000000e+00 0.0000000e+00 3.6760125e-01 ... 0.0000000e+00
  0.0000000e+00 3.1152647e-03]
 [0.0000000e+00 4.7619049e-03 1.4285714e-02 ... 0.0000000e+00
  0.0000000e+00 3.3333335e-02]
 [7.6423385e-05 3.3626289e-03 7.3290028e-02 ... 2.3691249e-03
  1.5437524e-02 0.0000000e+00]
 ...
 [1.0471204e-03 0.0000000e+00 9.1623040e-03 ... 2.6178011e-03
  7.8534032e-04 0.0000000e+00]
 [1.2026458e-04 1.0823812e-03 3.7161756e-02 ... 4.8105832e-04
  1.9819604e-01 9.6211664e-04]
 [5.6529115e-04 2.2611646e-03 1.9106840e-01 ... 0.0000000e+00
  2.9395139e-02 0.0000000e+00]]


In [ ]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,RBR,NNPS,JJ,$,NNP,MD,WP,",",NN,VBG,),DT,:,RB,JJR,IN,RP,VB,WP$,``,CC,VBZ,FW,PRP$,WRB,JJS,TO,UH,EX,NNS,'',PDT,(,VBD,WDT,RBS,VBP,VBN,.,SYM,#,PRP,CD,POS
RBR,0.000000,0.000000,0.367601,0.000000,0.003115,0.000000,0.000000,0.065421,0.000000,0.000000,0.000000,0.099688,0.009346,0.109034,0.000000,0.074766,0.000000,0.003115,0.000000,0.000000,0.015576,0.000000,0.000000,0.000000,0.000000,0.000000,0.003115,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009346,0.000000,0.000000,0.006231,0.021807,0.208723,0.000000,0.000000,0.000000,0.000000,0.003115
NNPS,0.000000,0.004762,0.014286,0.000000,0.042857,0.035714,0.014286,0.171429,0.076190,0.011905,0.000000,0.004762,0.016667,0.004762,0.002381,0.104762,0.000000,0.002381,0.000000,0.000000,0.078571,0.026190,0.000000,0.000000,0.000000,0.000000,0.030952,0.000000,0.000000,0.014286,0.002381,0.000000,0.000000,0.121429,0.000000,0.000000,0.071429,0.007143,0.107143,0.000000,0.000000,0.000000,0.000000,0.033333
JJ,0.000076,0.003363,0.073290,0.002216,0.033091,0.000611,0.000535,0.028200,0.458005,0.002140,0.000535,0.005885,0.002751,0.005273,0.000153,0.053726,0.000229,0.000382,0.000076,0.001605,0.016278,0.000688,0.000076,0.000306,0.000688,0.000306,0.029729,0.000000,0.000000,0.225067,0.004509,0.000076,0.000611,0.001528,0.000535,0.000153,0.000306,0.002751,0.026366,0.000000,0.000076,0.002369,0.015438,0.000000
$,0.000000,0.000000,0.004000,0.000000,0.000571,0.000000,0.000000,0.000000,0.009143,0.000571,0.000000,0.000000,0.000000,0.000571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.985143,0.000000
NNP,0.000352,0.011064,0.011165,0.000101,0.390113,0.010360,0.000553,0.139358,0.064826,0.002364,0.003068,0.002615,0.006689,0.007896,0.000201,0.039177,0.000000,0.001760,0.000000,0.001056,0.042144,0.035405,0.000654,0.000151,0.000956,0.000000,0.007141,0.000000,0.000000,0.026856,0.002163,0.000000,0.003118,0.062965,0.001257,0.000000,0.003370,0.000402,0.055421,0.000000,0.000000,0.000805,0.017200,0.047274
MD,0.000923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003692,0.000000,0.000000,0.000461,0.005538,0.000000,0.174435,0.000923,0.004153,0.000000,0.787264,0.000000,0.005076,0.000461,0.000000,0.000000,0.000461,0.000000,0.000000,0.004153,0.000000,0.000000,0.000923,0.000000,0.000000,0.000000,0.002307,0.000000,0.000000,0.000000,0.000461,0.001846,0.000000,0.000000,0.006922,0.000000,0.000000
WP,0.000000,0.000000,0.003781,0.000000,0.007561,0.069943,0.000000,0.001890,0.022684,0.000000,0.001890,0.049149,0.000000,0.047259,0.001890,0.009452,0.000000,0.001890,0.000000,0.003781,0.000000,0.215501,0.000000,0.003781,0.000000,0.000000,0.013233,0.000000,0.000000,0.015123,0.000000,0.000000,0.000000,0.264650,0.000000,0.000000,0.179584,0.000000,0.000000,0.000000,0.000000,0.085066,0.001890,0.000000
",",0.000557,0.000836,0.038997,0.001393,0.122563,0.008728,0.010492,0.000000,0.051625,0.044290,0.000000,0.133890,0.000093,0.045775,0.002321,0.096750,0.000093,0.004550,0.001764,0.013185,0.100186,0.032591,0.000093,0.003343,0.006128,0.000000,0.009471,0.000093,0.002786,0.028784,0.057382,0.000000,0.000093,0.052182,0.033519,0.001393,0.007707,0.022934,0.000000,0.000000,0.000000,0.044011,0.019220,0.000186
NN,0.003848,0.000299,0.010018,0.000498,0.009752,0.018178,0.002455,0.112482,0.117723,0.007895,0.002023,0.007762,0.011477,0.017050,0.001360,0.247753,0.000066,0.001426,0.000199,0.002189,0.039705,0.038777,0.000166,0.000166,0.001990,0.000000,0.041463,0.000000,0.000332,0.084950,0.005208,0.000033,0.001791,0.048628,0.007861,0.000199,0.003649,0.012107,0.105682,0.000100,0.000033,0.005274,0.005407,0.022025
VBG,0.003056,0.000917,0.064792,0.005807,0.039120,0.000000,0.002751,0.017421,0.129279,0.001834,0.000611,0.192237,0.001222,0.038509,0.008863,0.162286,0.006112,0.003362,0.000000,0.005807,0.009169,0.005501,0.000000,0.027812,0.001834,0.000000,0.089242,0.000000

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_tagged_words)) for x in range(10)]
 
test_run = [test_tagged_words[i] for i in rndom]
test_tagged_words=list(i[0] for i in test_run)


In [ ]:
#testing the accuracy with random 10 lines beacuse the whole is taking more than 2 hr to compile
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  14.918877840042114
Viterbi Algorithm Accuracy:  60.0


In [ ]:
#Code to test all the test sentences but please dont try to run it 
test_set = test_tagged_words
test_untagged_words = list(i[0] for i in test_set)


start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(test_tagged_words, tagged_seq) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)